# Automatic Speech Recognition (ASR) using Transformers

## Import libraries

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/studio-lab-
[nltk_data]     user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import torch
import torchaudio.functional as F
import os
import IPython

from datasets import load_dataset
from transformers import AutoModelForCTC, AutoProcessor

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



## Quick demo combining `Wav2Vec2`, `PyCTCDecode`, and `HuggingFace`
- https://twitter.com/PatrickPlaten/status/1468999507488788480

### Spanish language model

In [3]:
model_id = "patrickvonplaten/wav2vec2-large-xlsr-53-spanish-with-lm"

### Local speech & language model

In [4]:
model = AutoModelForCTC.from_pretrained(model_id)
processor = AutoProcessor.from_pretrained(model_id)

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/transformers/configuration_utils.py:348: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Loading the LM will be faster if you build a binary file.
Reading /home/studio-lab-user/.cache/pyctcdecode/patrickvonplaten__wav2vec2-large-xlsr-53-spanish-with-lm.main.5569ff8b50dc9207760d321264a6dfd18ee4324f/language_model/kenLM.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************


In [5]:
# load speech sample in streaming
sample = next(iter(load_dataset("common_voice", "es", split="test", streaming=True)))
resampled_audio = F.resample(torch.tensor(sample["audio"]["array"]), 48_000, 16_000).numpy()
input_values = processor(resampled_audio, return_tensors="pt", sampling_rate=16000).input_values

In [6]:
IPython.display.Audio(sample["audio"]["array"], rate=48000)

### Transcribe speech sample

In [7]:
with torch.no_grad():
    logits = model(input_values).logits
    transcription = processor.batch_decode(logits.numpy()).text
print(transcription)

['bien y qué regalo vas a abrir primero']


## Long-form audio transcription using Librosa
- https://github.com/huggingface/transformers/issues/10366#issuecomment-786780983

### Download audio from YouTube 

In [8]:
# !pip install --upgrade youtube-dl
# !pip install ffmpeg-python
# !pip install ffmpeg
# !pip install ffprobe

In [9]:
if not os.path.exists("data"):
    os.makedirs("data")

In [10]:
YouTubeID = 'Q8pI4m_-vDg'
OutputFile = 'data/raw_audio.mp3'

In [11]:
!youtube-dl -o $OutputFile $YouTubeID --restrict-filenames --extract-audio --audio-format mp3 

[youtube] Q8pI4m_-vDg: Downloading webpage
[download] Destination: data/raw_audio.mp3
[download] 100% of 1.26MiB in 00:2802KiB/s ETA 00:00
[ffmpeg] Post-process file data/raw_audio.mp3 exists, skipping


### Pre-process audio
- https://colab.research.google.com/gist/titu1994/a44fffd459236988ee52079ff8be1d2e/long-audio-transcription-citrinet.ipynb

In [12]:
import sys
import glob 
import subprocess

def transcode(input_dir, output_format, sample_rate, skip, duration):
    files = glob.glob(os.path.join(input_dir, "*.mp3"))

    # Filter out additional directories
    files = [f for f in files if not os.path.isdir(f)]

    output_dir = os.path.join(input_dir, "processed")

    if not os.path.exists(output_dir):
        print(f"Output directory {output_dir} does not exist, creating ...")
        os.makedirs(output_dir)

    for filepath in files:
        output_filename = os.path.basename(filepath)
        output_filename = os.path.splitext(output_filename)[0]

        output_filename = f"{output_filename}_processed.{output_format}"

        args = [
            'ffmpeg',
            '-i',
            str(filepath),
            '-ar',
            str(sample_rate),
            '-ac',
            str(1),
            '-y'
        ]

        if skip is not None:
            args.extend(['-ss', str(skip)])

        if duration is not None:
            args.extend(['-to', str(duration)])

        args.append(os.path.join(output_dir, output_filename))
        command = " ".join(args)
        !{command}

    print("\n")
    print(f"Finished trancoding {len(files)} audio files")

In [13]:
transcode(
        input_dir="./data/",
        output_format="wav",
        sample_rate=16000,
        skip=None,
        duration=None,
    )

Output directory ./data/processed does not exist, creating ...
ffmpeg version 4.4.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 9.4.0 (GCC)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1636205340875/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1636205340875/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-demuxer=dash --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-vaapi --enable-libx264 --enable-libx265 --enable-libaom --enable-libsvtav1 --enable-libxml2 --enable-libvpx --enable-pic --enable-pthreads --enable-shared --disable-static --enable-version3 --enable-zlib --enable-libmp3lame --pkg-config=/home/conda/feedstock_root/build

### Transcribe the processed audio file

In [14]:
import librosa
import soundfile as sf

In [15]:
def correct_sentence(input_text):
    sentences = nltk.sent_tokenize(input_text)
    return (' '.join([s.replace(s[0],s[0].capitalize(),1) for s in sentences]))

def asr_transcript(processor, model, input_file):
    transcript = ""
    # Ensure that the sample rate is 16k
    # data, samplerate = sf.read(input_file)
    samplerate = librosa.get_samplerate(input_file)

    # Stream over 30 seconds chunks rather than load the full file
    stream = librosa.stream(
        input_file,
        block_length=30,
        frame_length=16000,
        hop_length=16000
    )

    for speech in stream:
        if len(speech.shape) > 1:
            speech = speech[:, 0] + speech[:, 1]
            
        resampled_audio = F.resample(torch.tensor(speech), samplerate, samplerate).numpy()
        input_values = processor(resampled_audio, return_tensors="pt", sampling_rate=samplerate).input_values
            
        with torch.no_grad():
            logits = model(input_values).logits
            transcription = processor.batch_decode(logits.numpy()).text

        print(transcription)
        transcript += correct_sentence(transcription[0])

    return transcript

In [16]:
model_id = "patrickvonplaten/wav2vec2-large-xlsr-53-spanish-with-lm"
model = AutoModelForCTC.from_pretrained(model_id)
processor = AutoProcessor.from_pretrained(model_id)

Loading the LM will be faster if you build a binary file.
Reading /home/studio-lab-user/.cache/pyctcdecode/patrickvonplaten__wav2vec2-large-xlsr-53-spanish-with-lm.main.5569ff8b50dc9207760d321264a6dfd18ee4324f/language_model/kenLM.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************


In [17]:
audio_path = "data/processed/raw_audio_processed.wav"

In [18]:
IPython.display.Audio(filename=OutputFile, rate=48000)

In [19]:
transcript_result = asr_transcript(processor, model, audio_path)

['gorgescuales para usted la naturaleza del tiempo lo que yo querría saber lo que estaba tratando de saber toda mi vida el corro siempre que la sentencia tan justa de san agustín las confesiones qué es el tiempo si no me lo preguntan lo sé si no lo preguntan nora decit el sentía que era el tiempo no se definirlo lo dijo muy bien para el miss el voren esencial is la perplejidad']
['aque ese problema de ciencia personal si lo pienso en mí pero no soy solamente al que existe en este momento rasque en este momento lo puede ser cordura de ustedes y estamento los ecos de usted soy yo el gorgesemodo que yo soy tambien mi pasado pero eso pasado su mayor parte estorbidado sin embargo de algo que persiste estatual misterios tiempo no que se siente muy claro el caso de identidad personal']
['que hace que uno sea el mismo a lo largo del tiempo y sin embargo una olvidado a la mayor parte del pasado lo que se recuerda del pasado es mínimo y lo que servía casi todo cada diesmodora miles de percepcion

In [20]:
print(transcript_result)

Gorgescuales para usted la naturaleza del tiempo lo que yo querría saber lo que estaba tratando de saber toda mi vida el corro siempre que la sentencia tan justa de san agustín las confesiones qué es el tiempo si no me lo preguntan lo sé si no lo preguntan nora decit el sentía que era el tiempo no se definirlo lo dijo muy bien para el miss el voren esencial is la perplejidadAque ese problema de ciencia personal si lo pienso en mí pero no soy solamente al que existe en este momento rasque en este momento lo puede ser cordura de ustedes y estamento los ecos de usted soy yo el gorgesemodo que yo soy tambien mi pasado pero eso pasado su mayor parte estorbidado sin embargo de algo que persiste estatual misterios tiempo no que se siente muy claro el caso de identidad personalQue hace que uno sea el mismo a lo largo del tiempo y sin embargo una olvidado a la mayor parte del pasado lo que se recuerda del pasado es mínimo y lo que servía casi todo cada diesmodora miles de percepciones miles de 